In [1]:
!nvidia-smi

Tue Sep  2 06:46:31 2025       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.247.01             Driver Version: 535.247.01   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  NVIDIA GeForce RTX 4090        Off | 00000000:01:00.0  On |                  Off |
|  0%   42C    P5              40W / 450W |    583MiB / 24564MiB |     11%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

In [2]:
from transformers import pipeline

In [3]:
model_id = "openai/gpt-oss-20b"
pipe = pipeline(
    "text-generation",
    model=model_id,
    torch_dtype="auto",
    device_map="auto",
)

Fetching 40 files:   0%|          | 0/40 [00:00<?, ?it/s]

Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

Device set to use cuda:0


In [4]:
!nvidia-smi

Tue Sep  2 06:46:37 2025       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.247.01             Driver Version: 535.247.01   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  NVIDIA GeForce RTX 4090        Off | 00000000:01:00.0  On |                  Off |
|  0%   44C    P2              80W / 450W |  14144MiB / 24564MiB |      5%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

In [5]:
print(pipe.model)

GptOssForCausalLM(
  (model): GptOssModel(
    (embed_tokens): Embedding(201088, 2880, padding_idx=199999)
    (layers): ModuleList(
      (0-23): 24 x GptOssDecoderLayer(
        (self_attn): GptOssAttention(
          (q_proj): Linear(in_features=2880, out_features=4096, bias=True)
          (k_proj): Linear(in_features=2880, out_features=512, bias=True)
          (v_proj): Linear(in_features=2880, out_features=512, bias=True)
          (o_proj): Linear(in_features=4096, out_features=2880, bias=True)
        )
        (mlp): GptOssMLP(
          (router): GptOssTopKRouter()
          (experts): Mxfp4GptOssExperts()
        )
        (input_layernorm): GptOssRMSNorm((2880,), eps=1e-05)
        (post_attention_layernorm): GptOssRMSNorm((2880,), eps=1e-05)
      )
    )
    (norm): GptOssRMSNorm((2880,), eps=1e-05)
    (rotary_emb): GptOssRotaryEmbedding()
  )
  (lm_head): Linear(in_features=2880, out_features=201088, bias=False)
)


In [6]:
parameters_state_dict = pipe.model.state_dict()
for key, value in parameters_state_dict.items():
    print(key, value.size(), value.dtype)

model.embed_tokens.weight torch.Size([201088, 2880]) torch.bfloat16
model.layers.0.self_attn.sinks torch.Size([64]) torch.bfloat16
model.layers.0.self_attn.q_proj.weight torch.Size([4096, 2880]) torch.bfloat16
model.layers.0.self_attn.q_proj.bias torch.Size([4096]) torch.bfloat16
model.layers.0.self_attn.k_proj.weight torch.Size([512, 2880]) torch.bfloat16
model.layers.0.self_attn.k_proj.bias torch.Size([512]) torch.bfloat16
model.layers.0.self_attn.v_proj.weight torch.Size([512, 2880]) torch.bfloat16
model.layers.0.self_attn.v_proj.bias torch.Size([512]) torch.bfloat16
model.layers.0.self_attn.o_proj.weight torch.Size([2880, 4096]) torch.bfloat16
model.layers.0.self_attn.o_proj.bias torch.Size([2880]) torch.bfloat16
model.layers.0.mlp.router.weight torch.Size([32, 2880]) torch.bfloat16
model.layers.0.mlp.router.bias torch.Size([32]) torch.bfloat16
model.layers.0.mlp.experts.gate_up_proj_bias torch.Size([32, 5760]) torch.bfloat16
model.layers.0.mlp.experts.down_proj_bias torch.Size([32

In [7]:
messages = [
    {"role": "user", "content": "Explain quantum mechanics clearly and concisely."},
]

outputs = pipe(
    messages,
    max_new_tokens=1024,
)
print(outputs[0]["generated_text"][-1]["content"])

analysisWe need to explain quantum mechanics clearly and concisely. Should cover core principles: wave-particle duality, superposition, uncertainty principle, quantization, measurement, wavefunction, Schrödinger equation, entanglement, etc. Keep it concise, but thorough enough. Probably use analogies. Also maybe mention applications. Use simple language. Provide structure: introduction, key concepts, key equations, interpretation, conclusion. Probably aim for ~500-700 words. Keep it understandable.assistantfinal**Quantum Mechanics in a nutshell**

| What | Why it matters | How we see it |
|------|----------------|---------------|
| **Particles are also waves** | Explains diffraction, interference, and the “spooky” behavior of atoms. | A single electron shot through a double‑slit creates an interference pattern, just like water waves. |
| **Superposition** | A quantum system can be in many states at once until we look. | A single photon can be in two places at once; only when measured d

In [8]:
messages = [
    {
        "role": "user",
        "content": "how much memory to reserve for gpt oss on a 4090 rtx gpu",
    },
]

outputs = pipe(
    messages,
    max_new_tokens=1024,
)
print(outputs[0]["generated_text"][-1]["content"])

analysisThe question: "how much memory to reserve for gpt oss on a 4090 rtx gpu". They likely refer to GPT-OSS? Maybe "GPT-OSS" is an open source version of GPT? They want to know how much memory to reserve on a RTX 4090 GPU. They might be asking about memory allocation for GPT models, maybe for inference or training. The RTX 4090 has 24GB of GDDR6X memory. They might want to know how much memory to reserve for GPT-OSS? They might be on a server with multiple GPU usage. They might need to reserve some memory for system overhead. They might want to know "how much memory to reserve for GPT-OSS" i.e., how much memory will GPT-OSS use? Perhaps they want to know how many tokens per batch, etc.

Need to answer: The memory usage depends on model size, batch size, sequence length, precision (FP16, BF16, INT8, etc.), caching for attention, etc. For GPT-OSS (maybe GPT-2 or GPT-3?). GPT-OSS might refer to "GPT Open Source" like GPT-Neo, GPT-J, GPT-NeoX, GPT-4? But GPT-4 is not open source. GPT-OS

In [9]:
messages = [
    {
        "role": "user",
        "content": "Can you show me a Python code example for a simple web server?",
    },
]

outputs = pipe(
    messages,
    max_new_tokens=1024,
)
print(outputs[0]["generated_text"][-1]["content"])

analysisWe need to respond with a Python code example for a simple web server. Likely using built-in http.server or Flask or FastAPI. The question is generic: "Can you show me a Python code example for a simple web server?" Provide a minimal example. Use built-in http.server for simplicity. Or use Flask for demonstration. Provide explanation. Also maybe mention usage. Provide code block. Provide a brief explanation. That's it.assistantfinalSure! Below are two quick ways to spin up a simple web server in Python:

| Approach | What you’ll get | How to run |
|----------|-----------------|------------|
| **Built‑in `http.server`** | A tiny static file server that serves whatever is in your current directory. | `python3 -m http.server 8000` |
| **Flask** | A lightweight web framework that lets you add routes, templates, etc. | `python3 -m flask run` (after installing Flask) |

---

## 1. Using the built‑in `http.server`

```python
# server_builtin.py
import http.server
import socketserver



In [10]:
messages = [
    {
        "role": "user",
        "content": "Write some pytorch code an intersection across two sparse tensors. Sparse tensors are defined as a tuple of 2 tensors : first tensor is lengths of values, the second tensor is coallesced tensor with all values. THe length of first tensor is the batch_size. DO NOT loop over the batch_size. do batched operations as vectorized directly in pytorch",
    },
]

outputs = pipe(
    messages,
    max_new_tokens=4096,
)
print(outputs[0]["generated_text"][-1]["content"])

analysisWe need to interpret the question: We have two sparse tensors defined as a tuple of 2 tensors: first tensor is lengths of values, second tensor is coalesced tensor with all values. The length of first tensor is batch_size. We need to compute intersection across two sparse tensors. Intersection presumably means element-wise intersection of indices? Since we don't have indices explicitly, only lengths and values? But maybe the sparse tensors represent variable-length sequences of values, like ragged tensors. The intersection across two sparse tensors for each batch element: find common values between the two sequences? So intersection per batch. Need to do vectorized across batch. For each batch index, we have two sequences: values1[b], values2[b], lengths l1[b], l2[b]. We need intersection of sets? Or intersection of index positions? Likely intersection of values.

Given we can't loop over batch size, we need to use broadcasting and masking. Approach: pad sequences to max length

In [11]:
import torch

messages = [
    {"role": "user", "content": "Explain how neural networks process text data."},
]


prof = torch.profiler.profile(
    activities=[
        torch.profiler.ProfilerActivity.CPU,
        torch.profiler.ProfilerActivity.CUDA,
    ],
    schedule=torch.profiler.schedule(wait=1, warmup=1, active=1, repeat=1),
    on_trace_ready=torch.profiler.tensorboard_trace_handler("./tensorboard"),
    # used when outputting for tensorboard
)
prof.start()

for i in range(5):
    # Run the model inference
    outputs = pipe(
        messages,
        max_new_tokens=64,  # Using smaller token count for profiling
    )
    prof.step()
prof.stop()
print("\n--- Profiler Results (Table) ---")
print(prof.key_averages().table(sort_by="cuda_time_total", row_limit=100))


--- Profiler Results (Table) ---
-------------------------------------------------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  
                                                   Name    Self CPU %      Self CPU   CPU total %     CPU total  CPU time avg     Self CUDA   Self CUDA %    CUDA total  CUDA time avg    # of Calls  
-------------------------------------------------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  
                                          ProfilerStep*         0.00%       0.000us         0.00%       0.000us       0.000us        2.393s       454.50%        2.393s        2.393s             1  
                                          ProfilerStep*        43.40%        1.039s       100.00%        2.394s        2.394s       0.000us         0.00%     342.903ms     3